In [1]:
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append("..")
os.environ["AUTOGEN_USE_DOCKER"] = "False"
import time

In [ ]:
from agents.langchian_master_agent import MasterAgent

In [3]:
master_agent = MasterAgent()

In [ ]:
from agents.entity_recognition import EntityRecognitionAgent
from agents.query_refinement import QueryRefinementAgent
from models.embeddings import BiEncoderModel, CrossEncoderModel
from models.entity_recognition import GLiNERModel
from database.qdrant_client import QdrantClient
from database.qdrant_client import QdrantClient

2025-04-18 01:51:21,247 | INFO | nl-to-sparql | Logging initialized: console=INFO, file=DEBUG


In [5]:
from agents.plan_formulation_2 import PlanFormulationAgent
from agents.validation_2 import ValidationAgent
from agents.response_generation_2 import ResponseGenerationAgent

In [6]:
bi_encoder = BiEncoderModel(
        model_name_or_path="sentence-transformers/all-MiniLM-L6-v2"
    )
    
# Initialize Cross-Encoder for reranking
cross_encoder = CrossEncoderModel(
    model_name_or_path="cross-encoder/ms-marco-MiniLM-L-6-v2"
)

# Initialize entity recognition model
entity_recognition_model = GLiNERModel(
    model_name_or_path="urchade/gliner_medium-v2.1"
)

2025-04-18 01:51:22,370 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cuda:0
2025-04-18 01:51:22,371 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-04-18 01:51:30,610 | INFO | sentence_transformers.cross_encoder.CrossEncoder | Use pytorch device: cuda:0


Fetching 5 files: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 26181.67it/s]


Loaded GLiNER model on cuda


In [7]:
qdrant_client = QdrantClient(url=os.getenv("QDRANT_URL"))
query_refinement_agent = QueryRefinementAgent(
    qdrant_client=qdrant_client,
    embedding_model=bi_encoder
)

entity_recognition_agent = EntityRecognitionAgent(
    entity_recognition_model=entity_recognition_model, 
    ontology_store=None
)

2025-04-18 01:51:42,154 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cuda:0
2025-04-18 01:51:42,155 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-18 01:51:45,041 | INFO | httpx | HTTP Request: GET http://172.26.33.199:2513 "HTTP/1.1 200 OK"


In [8]:
plan_formulation_agent = PlanFormulationAgent()
validation_agent = ValidationAgent()
response_generation_agent = ResponseGenerationAgent()

2025-04-18 01:51:45,398 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cuda:0
2025-04-18 01:51:45,400 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-04-18 01:51:48,490 | INFO | httpx | HTTP Request: GET http://172.26.33.199:2513 "HTTP/1.1 200 OK"


In [9]:
master_agent.register_slave_agent("query_refinement", query_refinement_agent)
master_agent.register_slave_agent("entity_recognition", entity_recognition_agent)
master_agent.register_slave_agent("plan_formulation", plan_formulation_agent)
master_agent.register_slave_agent("validation", validation_agent)
master_agent.register_slave_agent("response_generation", response_generation_agent)

In [10]:
result = master_agent.process_query(
    "What is the duration of Trieu Dynasty?",
    []
)

2025-04-18 01:51:48.515 | INFO     | agents.master_agent_2:process_query:53 - Processing query: What is the duration of Trieu Dynasty?
Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.74it/s]

2025-04-18 01:51:48,801 | INFO | httpx | HTTP Request: POST http://172.26.33.199:2513/collections/refinement_examples/points/query "HTTP/1.1 200 OK"
QueryRefinementProxy (to QueryRefinementAgent):

I need you to refine the following user query about a knowledge graph or ontology into a clear, standalone question.
Use the conversation history to fill in any missing context, resolve references, 
and make sure the query is complete and unambiguous.

Current Conversation History:


User Query: What is the duration of Trieu Dynasty?

Similar Examples of Query Refinement:


Please provide the refined query as a standalone question. 
Do not include explanations or additional text, just the refined query.
Ensure the refined query would make sense even without the conversation history.
Include specific entity names whenever they're referenced directly or indirectly.


--------------------------------------------------------------------------------
QueryRefinementAgent (to QueryRefinementProxy):


2025-04-18 01:51:48.834 | INFO     | agents.master_agent_2:process_query:60 - Refined query: What is the duration of the Trieu Dynasty in history?
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2025-04-18 01:51:49.209 | INFO     | agents.master_agent_2:process_query:67 - Recognized 2 entities
2025-04-18 01:51:49.211 | INFO     | agents.master_agent_2:process_query:80 - Mapped entities to ontology terms


2025-04-18 01:51:51,333 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2025-04-18 01:51:51.353 | INFO     | agents.master_agent_2:process_query:85 - Create plan for SPARQL query successfully: [{'step': 'Find the start date of the Trieu Dynasty', 'sparql_type': 'SELECT', 'level': 'simple'}, {'step': 'Find the end date of the Trieu Dynasty', 'sparql_type': 'SELECT', 'level': 'simple'}, {'step': 'Compute the duration of the Trieu Dynasty using the start and end dates', 'sparql_type': 'SELECT', 'level': 'complex'}]


...

2025-04-18 01:51:54.663 | INFO     | agents.master_agent_2:process_query:90 - Validation result: {'is_valid': False, 'feedback': [{'step': 'Find the start date of the Trieu Dynasty', 'feedback': 'This step is correct, but ensure the SPARQL query retrieves the correct property for the start date.'}, {'step': 'Find the end date of the Trieu Dynasty', 'feedback': 'This step is also correct, but ensure the SPARQL query retrieves the correct property for the end date.'}, {'step': 'Compute the duration of the Trieu Dynasty using the start and end dates', 'feedback': 'This step is valid, but it should be noted that SPARQL can directly compute the duration in the SELECT query if the start and end dates are retrieved in the same query.'}]}


2025-04-18 01:51:57,109 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2025-04-18 01:51:57.114 | INFO     | agents.master_agent_2:process_query:96 - Fixed plan successfully: [{'step': 'Find the start date of the Trieu Dynasty in history', 'sparql_type': 'SELECT', 'level': 'simple'}, {'step': 'Find the end date of the Trieu Dynasty in history', 'sparql_type': 'SELECT', 'level': 'simple'}, {'step': 'Calculate the duration of the Trieu Dynasty using the start and end dates', 'sparql_type': 'SELECT', 'level': 'complex'}]


2025-04-18 01:52:00,093 | INFO | httpx | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


2025-04-18 01:52:00.097 | INFO     | agents.master_agent_2:process_query:99 - Validation result: {'is_valid': False, 'feedback': [{'step': 'Find the start date of the Trieu Dynasty in history', 'feedback': 'This step is correct, but ensure the SPARQL query retrieves the correct property for the start date.'}, {'step': 'Find the end date of the Trieu Dynasty in history', 'feedback': 'This step is also correct, but ensure the SPARQL query retrieves the correct property for the end date.'}, {'step': 'Calculate the duration of the Trieu Dynasty using the start and end dates', 'feedback': 'This step is valid, but it should be noted that SPARQL can perform calculations directly in the SELECT query, so consider integrating this calculation into the previous steps to streamline the query.'}]}
2025-04-18 01:52:00.098 | INFO     | agents.master_agent_2:process_query:108 - Generated error response due to validation failure


In [11]:
result["response"]

"I'm sorry, but I couldn't create a valid SPARQL query for your question. [{'step': 'Find the start date of the Trieu Dynasty in history', 'feedback': 'This step is correct, but ensure the SPARQL query retrieves the correct property for the start date.'}, {'step': 'Find the end date of the Trieu Dynasty in history', 'feedback': 'This step is also correct, but ensure the SPARQL query retrieves the correct property for the end date.'}, {'step': 'Calculate the duration of the Trieu Dynasty using the start and end dates', 'feedback': 'This step is valid, but it should be noted that SPARQL can perform calculations directly in the SELECT query, so consider integrating this calculation into the previous steps to streamline the query.'}]"

In [12]:
What is the end date of Trieu Dynasty?
What is the start date of Trieu Dynasty?
What is the duration of Trieu Dynasty?
{'query': 'SELECT (xsd:integer(STR(?endYear)) - xsd:integer(STR(?startYear)) AS ?duration) WHERE { <https://CHeVIE.vn/ontologies/TrieuRulingDynastyStartDateDescription> <http://www.w3.org/2006/time#year> ?startYear . <https://CHeVIE.vn/ontologies/TrieuRulingDynastyEndDateDescription> <http://www.w3.org/2006/time#year> ?endYear . }',
   'step': 'Compute the duration of the Trieu Dynasty using the start and end dates'}]}

SyntaxError: unmatched ']' (68424690.py, line 5)